# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
1.3131e19,"""10427748905776168611""","""../../../../data/SampleHDS.jso…","""17291703901795668642""","""16204010219352260203""","""7066813686078969216""",false
1.4719e19,"""4333625867847959570""","""../../../../data/SampleHDS.jso…","""5037566824653361147""","""16204010219352260203""","""15109119096250725519""",true
8.8131e18,"""13123922201475826831""","""../../../../data/SampleHDS.jso…","""11793055027117407356""","""17993634590654756474""","""7066813686078969216""",false
2.7200e17,"""4948398817395327837""","""../../../../data/SampleHDS.jso…","""5037566824653361147""","""12585168333134380881""","""7066813686078969216""",false
8.4799e18,"""17186518998262780710""","""../../../../data/SampleHDS.jso…","""17291703901795668642""","""12585168333134380881""","""8202589507181348374""",true
1.3658e19,"""13969546352197067123""","""../../../../data/SampleHDS.jso…","""11793055027117407356""","""12963883698870176432""",null,false
3.8119e18,"""10304849338065023224""","""../../../../data/SampleHDS.jso…","""11793055027117407356""","""12585168333134380881""","""8202589507181348374""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_City': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
2.4189e18,"""6711345464731672664""","""../../../../data/SampleHDS.jso…","""17806548581483879004""","""FirstMortgage30yr""","""15373809272803060537""","""Rejected"""
4.6557e18,"""2647650755118811860""","""../../../../data/SampleHDS.jso…","""4811996235887263709""","""FirstMortgage30yr""","""6826099097547785329""","""Accepted"""
3.4675e18,"""15309630591406033599""","""../../../../data/SampleHDS.jso…","""4702127689482341553""","""MoneyMarketSavingsAccount""","""15373809272803060537""","""Rejected"""
1.8250e19,"""1445209891722146069""","""../../../../data/SampleHDS.jso…","""4811996235887263709""","""BasicChecking""","""15373809272803060537""","""Rejected"""
3.0686e18,"""3996814666541139240""","""../../../../data/SampleHDS.jso…","""17806548581483879004""","""BasicChecking""","""2378608845708450564""","""Accepted"""
1.8273e19,"""8009703105116097549""","""../../../../data/SampleHDS.jso…","""4702127689482341553""","""UPlusFinPersonal""",null,"""Rejected"""
1.7207e19,"""6278427285272900714""","""../../../../data/SampleHDS.jso…","""4702127689482341553""","""BasicChecking""","""2378608845708450564""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
2.8383e18,"""5053670615412188111""","""../../../../data/SampleHDS.jso…","""12992809753310160171""","""FirstMortgage30yr""","""2587185929665586991""","""Rejected"""
4.1508e18,"""2032736856086413460""","""../../../../data/SampleHDS.jso…","""13308523693436356107""","""FirstMortgage30yr""","""9773456977901110530""","""Accepted"""
7.5495e18,"""1270230512425409364""","""../../../../data/SampleHDS.jso…","""11430199213989519547""","""MoneyMarketSavingsAccount""","""2587185929665586991""","""Rejected"""
1.6746e19,"""14497146773705335566""","""../../../../data/SampleHDS.jso…","""13308523693436356107""","""BasicChecking""","""2587185929665586991""","""Rejected"""
1.2184e18,"""9811861286252726497""","""../../../../data/SampleHDS.jso…","""12992809753310160171""","""BasicChecking""","""4172838066194248138""","""Accepted"""
1.3271e19,"""15556177180209701514""","""../../../../data/SampleHDS.jso…","""11430199213989519547""","""UPlusFinPersonal""",null,"""Rejected"""
1.7361e19,"""16121661315912309018""","""../../../../data/SampleHDS.jso…","""11430199213989519547""","""BasicChecking""","""4172838066194248138""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_City=PREDICTOR_1
filename=filename
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
6.3776e18,"""1479990386607289885""","""7587988981258368701""","""FirstMortgage30yr""","""16776571533984735091""","""Rejected"""
2.9754e18,"""10483053782656404407""","""9559023664706771799""","""FirstMortgage30yr""","""4790523414832774907""","""Accepted"""
6.7822e18,"""5394710640613084763""","""11648770440733316472""","""MoneyMarketSavingsAccount""","""16776571533984735091""","""Rejected"""
1.7196e19,"""433181250249899939""","""9559023664706771799""","""BasicChecking""","""16776571533984735091""","""Rejected"""
1.4849e19,"""4655689389101676244""","""7587988981258368701""","""BasicChecking""","""5410920482003366391""","""Accepted"""
6.0123e18,"""9881330624229775634""","""11648770440733316472""","""UPlusFinPersonal""",null,"""Rejected"""
1.6584e19,"""17990650103828231291""","""11648770440733316472""","""BasicChecking""","""5410920482003366391""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'